In [15]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.cluster import MeanShift
import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
data = pd.read_csv('checkins.dat',sep='|').replace(r'^\s+$', np.nan, regex=True).dropna()
print data.shape
data.head()

(396634, 6)


,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18


In [59]:
d = pd.DataFrame()
d['latitude'] = data[data.columns[3]].apply(lambda x: float(x))
d['longitude'] = data[data.columns[4]].apply(lambda x: float(x))
print d.shape
d.head()

(396634, 2)


,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037


In [16]:
ms = MeanShift(bandwidth=0.1)

In [66]:
%%time
ms.fit(d[:100000])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [67]:
len(ms.cluster_centers_)

3229

In [69]:
print d[:100000].shape
d[:100000].head()

(100000, 2)


,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207
10,40.764462,-111.904565
11,33.448377,-112.074037


In [72]:
len(ms.labels_)

100000

In [77]:
d1 = d[:100000].copy()

In [79]:
d1['label'] = ms.labels_

In [120]:
ms.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ..., 
       [  39.2190608 , -121.0610606 ],
       [  -4.5585849 ,  105.4068079 ],
       [  46.7312745 , -117.1796158 ]])

In [121]:
clusters = pd.DataFrame()
clusters['count_el'] = d1.label.value_counts()
clusters['cluster_center_latitude'] = ms.cluster_centers_[:,0]
clusters['cluster_center_longitude'] = ms.cluster_centers_[:,1]

In [122]:
clusters.head()

,count_el,cluster_center_latitude,cluster_center_longitude
0,12506,40.717716,-73.991835
1,4692,33.449438,-112.002140
2,3994,33.446380,-111.901888
4,3526,41.878244,-87.629843
3,3363,37.688682,-122.409330


In [110]:
office = pd.DataFrame(data=[[33.751277, -118.188740],[25.867736, -80.324116],[51.503016, -0.075479],[52.378894, 4.885084],
                      [39.366487, 117.036146],[-33.868457, 151.205134]], columns=['latitude','longitude'])

In [111]:
office

,latitude,longitude
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


In [125]:
def dist_nearest_office(x):
    m = 1e6
    for i in office.values:
        dist = sc.spatial.distance.euclidean(x,i)
        if dist<m:
            m=dist
    return m

In [127]:
clusters['dist_nearest_office'] = [dist_nearest_office(i) for i in clusters.values[:,1:3]]

In [130]:
ans = clusters[clusters.dist_nearest_office == clusters.dist_nearest_office.min()]
ans

,count_el,cluster_center_latitude,cluster_center_longitude,dist_nearest_office
430,27,-33.86063,151.204776,0.007835


In [133]:
ans[['cluster_center_latitude','cluster_center_longitude']].to_csv('answer.csv',sep=' ',index=False, header=False)